In [1]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

filename = 'data/Head-and-Neck_04.mat'

f = h5py.File(filename)
print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))

Patient: Head-and-Neck 04


In [2]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  PTV 0-46 Gy -> (5120, 10125) (sparse)
  Spinal Cord -> (3775, 10125) (sparse)
  Brainstem -> (3137, 10125) (sparse)
  PTV Shell 15 mm -> (4991, 10125) (sparse)
  PTV Shell 30 mm -> (4986, 10125) (sparse)
  PTV Shell 40 mm -> (4991, 10125) (sparse)
  PTV Shell 5 mm -> (4936, 10125) (sparse)
  Patient -> (11887, 10125) (sparse)
  PTV Shell 0 mm -> (5083, 10125) (sparse)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  Cochlea (R) -> (64, 10125) (sparse)
  Cochlea (L) -> (63, 10125) (sparse)
  Parotid (R) -> (3875, 10125) (sparse)
  Parotid (R) (mean) -> Ignored
  Parotid (L) -> (4241, 10125) (sparse)
  Parotid (L) (mean) -> Ignored
  SMG (R) -> (1571, 10125) (sparse)
  SMG (R) (mean) -> Ignored
  SMG (L) -> (2142, 10125) (sparse)
  SMG (L) (mean) -> Ignored
  Oral Cavity -> (5303, 10125) (sparse)
  Oral Cavity (mean) -> Ignored
  External Ring 20 mm -> (5677, 10125) (sparse)
  Larynx -> (5339, 10125) (sparse)
  Larynx (mean) -> Ignore

In [3]:
def compute_dose(fluence):
    for name, roi in rois.items():
        roi.compute_dose(fluence)

def dose_table():
    print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))
    print()
    print(colored('{:20s} {:>9s} {:>9s} {:>9s}'.format('Region of Interest', 'Min.', 'Mean', 'Max.'), attrs=["bold"]))
    print(colored('-'*50, 'blue'))

    for name, roi in rois.items():
        print('{:20s} {:9.2f} {:9.2f} {:9.2f}'.format(name, roi.min, roi.mean, roi.max))

fluence = f['solutionX'][:].ravel()
compute_dose(fluence)
dose_table()

Patient: Head-and-Neck 04

Region of Interest        Min.      Mean      Max.
--------------------------------------------------
Patient                   0.00     26.76     48.30
Spinal Cord               0.00     18.28     28.22
Parotid (R)               0.24     18.25     48.30
Parotid (L)               0.75     20.00     48.30
SMG (R)                  22.47     39.43     48.30
SMG (L)                  44.84     46.20     48.30
MCS                      21.76     34.50     48.30
MCM                      19.61     40.31     48.30
MCI                      30.65     45.97     48.30
MCP                      21.39     37.85     48.30
Oesophagus               21.39     33.81     48.05
Brainstem                 0.00      2.64     18.30
Oral Cavity               9.63     27.40     48.30
Larynx                   16.93     41.65     48.30
PTV 0-46 Gy              43.07     47.49     48.30
Cochlea (R)               0.07      0.12      0.15
Cochlea (L)               4.19      6.27      8.15
PTV 

In [4]:
from enum import Enum
from dataclasses import dataclass

class CostFunction(Enum):
    Linear = 1
    Quadratic = 2
    gEUD = 3
    LTCP = 4
    DVH = 5
    Chain = 6

class Function:
    def __init__(self, name, type, is_constraint, is_minimize, target, is_scalar, priority):
        self.name = name
        self.type = CostFunction(type)
        self.minimum = None
        self.mean = None
        self.maximum = None
        self.target = target
        self.priority = priority

        if is_constraint:
            self.kind = 'Constraint'
        else:
            self.kind = 'Objective'

        if self.type.name == 'Linear':
            if self.kind == 'Objective':
                if is_scalar and is_minimize:
                    self.direction = 'mean'
                else:
                    self.direction = 'maximum'
            else:
                self.direction = 'maximum' if is_minimize else 'minimum'
        else:
            self.direction = ''

    def __repr__(self):
        return '{:20s} {} {:16s} {:>3d} {:8.2f}'.format(self.name, 
            colored('Const.', 'yellow') if self.kind == 'Constraint' else colored('Objec.', 'cyan'), 
            (self.type.name + ' (' + self.direction + ')') if self.direction != '' else self.type.name,
            self.priority, self.target)

def load_problem():
    print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))
    print()
    print(colored('Region of Interest   Kind   Function         Pr.   Target  Current', attrs=["bold"]))
    print(colored('------------------------------------------------------------------', 'blue'))
    
    for i in range(f['problem']['dataID'].shape[0]):
        name = decode_str(f[f['problem']['Name'][i][:][0]][:])
        type = f[f['problem']['Type'][i][0]][0][0]
        is_constraint = f[f['problem']['IsConstraint'][i][0]][0][0] == 1
        is_minimize = f[f['problem']['Minimise'][i][0]][0][0] == 1
        target = f[f['problem']['Objective'][i][0]][0][0]
        dataID = int(f[f['problem']['dataID'][i][0]][0][0]) - 1
        is_scalar = isinstance(f[f['data']['matrix']['A'][dataID,0]], h5py.Dataset)
        #is_sufficient = isinstance(f[f['problem']['Sufficient'][i][0]][0], np.ndarray)
        priority = int(f[f['problem']['Priority'][i][0]][0][0])
              
        function = Function(name, type, is_constraint, is_minimize, target, is_scalar, priority)
        print(function, end='')

        if name in rois:
            roi = rois[name]
            if function.type == CostFunction.Linear:
                if function.direction == 'minimum':
                    value = roi.min
                    if value > function.target:
                        color = 'green'
                    else:
                        color = 'red'
                elif function.direction == 'mean':
                    value = roi.mean
                    if value < function.target:
                        color = 'green'
                    else:
                        color = 'red'
                else:
                    value = roi.max
                    if value < function.target:
                        color = 'green'
                    else:
                        color = 'red'
                print(colored(' {:8.2f}'.format(value), color))
            else:
                print()
        else:
            print()
        
load_problem()

Patient: Head-and-Neck 04

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30    48.30
Spinal Cord          Const. Linear (maximum)   0    38.00    28.22
Brainstem            Const. Linear (maximum)   0    38.00    18.30
Patient              Const. Linear (maximum)   0    48.30    48.30
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00    46.00
Cochlea (R)          Const. Linear (maximum)   0    30.00     0.15
Cochlea (L)          Const. Linear (maximum)   0    30.00     8.15
Parotid (R)          Const. Linear (maximum)   0    48.30    48.30
Parotid (L)          Const. Linear (maximum)   0    48.30    48.30
SMG (R)              Const. Linear (maximum)   0    48.30    48.30
SMG (L)              Const. Linear (maximum)   0    48.30    48.30
Oral Cavity          Const. Linear (maximum)   0    48.30    48.30
Larynx               Const. Linear 

In [5]:
with open('../Code_POO/multicore/results_adam/Head_and_Neck_04/x_20250102_101756.txt') as file: 
    fluence = [float(x.split()[-1]) for x in file.readlines()]
compute_dose(fluence)

In [6]:
load_problem()

Patient: Head-and-Neck 04

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30    48.67
Spinal Cord          Const. Linear (maximum)   0    38.00    24.44
Brainstem            Const. Linear (maximum)   0    38.00    20.38
Patient              Const. Linear (maximum)   0    48.30    48.69
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00    49.09
Cochlea (R)          Const. Linear (maximum)   0    30.00     1.27
Cochlea (L)          Const. Linear (maximum)   0    30.00     7.57
Parotid (R)          Const. Linear (maximum)   0    48.30    47.31
Parotid (L)          Const. Linear (maximum)   0    48.30    46.95
SMG (R)              Const. Linear (maximum)   0    48.30    46.50
SMG (L)              Const. Linear (maximum)   0    48.30    47.16
Oral Cavity          Const. Linear (maximum)   0    48.30    47.75
Larynx               Const. Linear 

In [33]:
with open('../Code_POO/multicore/results_gradient/Head_and_Neck_02/x_20241230_131536.txt') as file: 
    fluence = [float(x.split()[-1]) for x in file.readlines()]
compute_dose(fluence)

In [34]:
load_problem()

Patient: Head-and-Neck 02

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30    48.35
Spinal Cord          Const. Linear (maximum)   0    38.00    24.96
Brainstem            Const. Linear (maximum)   0    38.00     9.45
Patient              Const. Linear (maximum)   0    48.30    48.28
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00    47.32
Parotid (R)          Const. Linear (maximum)   0    48.30    47.64
Parotid (L)          Const. Linear (maximum)   0    48.30    47.10
SMG (R)              Const. Linear (maximum)   0    48.30    46.50
SMG (L)              Const. Linear (maximum)   0    48.30    46.83
Oral Cavity          Const. Linear (maximum)   0    48.30    42.74
Larynx               Const. Linear (maximum)   0    48.30    47.79
MCS                  Const. Linear (maximum)   0    48.30    45.85
MCM                  Const. Linear 